In [ ]:
# Instalando as bibliotecas necessárias (execute apenas se ainda não estiverem instaladas)
!pip install osmnx networkx folium geopy shapely scikit-learn geojson pyproj

In [ ]:
import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import time
import warnings
import sys
import geopandas as gpd
import geojson
from pyproj import Transformer

# Ignorar FutureWarnings temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

def geocode_address(address):
    """
    Converte um endereço em coordenadas geográficas (latitude e longitude).
    """
    geolocator = Nominatim(user_agent="shortest_path_tester")
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        raise ValueError("Endereço não encontrado.")

def get_pizza_pois(G_projected, origin_point_geo, radius=5000, cuisine='pizza'):
    """
    Busca pontos de interesse (POIs) de pizzarias próximas à origem.
    """
    tags = {'amenity': 'restaurant', 'cuisine': cuisine}
    pizzas = ox.geometries_from_point(origin_point_geo, tags=tags, dist=radius)
    
    if pizzas.empty:
        print("Nenhuma pizzaria encontrada na área.")
        return [], []
    else:
        print(f"{len(pizzas)} pizzarias encontradas.")
        
        # Certifique-se de que a coluna 'geometry' existe
        if 'geometry' not in pizzas.columns:
            print("Nenhuma geometria encontrada para as pizzarias.")
            return [], []
        
        # Reprojetar para o CRS do grafo projetado
        try:
            pizzas_projected = pizzas.to_crs(G_projected.graph['crs'])
        except Exception as e:
            print(f"Erro ao reprojetar geometrias: {e}")
            return [], []
        
        # Explodir geometrias compostas (MultiPolygon, etc.) em geometrias simples
        try:
            pizzas_projected = pizzas_projected.explode(index_parts=False).reset_index(drop=True)
        except Exception as e:
            print(f"Erro ao explodir geometrias: {e}")
            return [], []
        
        # Remover geometrias inválidas
        pizzas_projected = pizzas_projected[pizzas_projected.is_valid]
        
        # Filtrar apenas geometrias do tipo Polygon ou MultiPolygon
        pizzas_projected = pizzas_projected[pizzas_projected.geometry.type.isin(['Polygon', 'MultiPolygon'])]
        
        if pizzas_projected.empty:
            print("Nenhuma pizzaria válida encontrada após filtragem.")
            return [], []
        
        # Calcular os centróides em CRS projetado
        pizzeria_centroids_projected = pizzas_projected.geometry.centroid
        
        # Remover quaisquer centróides inválidos
        pizzeria_centroids_projected = pizzeria_centroids_projected[pizzeria_centroids_projected.is_valid]
        
        if pizzeria_centroids_projected.empty:
            print("Nenhum centróide válido calculado.")
            return [], []
        
        # Extrair coordenadas projetadas
        pizzeria_coords_proj = [(point.x, point.y) for point in pizzeria_centroids_projected]
        
        # Encontrar os nós mais próximos no grafo projetado
        pizzeria_nodes = ox.distance.nearest_nodes(
            G_projected, 
            X=[coord[0] for coord in pizzeria_coords_proj], 
            Y=[coord[1] for coord in pizzeria_coords_proj]
        )
        
        # Converter os centróides projetados para CRS geográfico para visualização usando GeoPandas
        try:
            pizzeria_centroids_geo = pizzeria_centroids_projected.to_crs(epsg=4326)
        except Exception as e:
            print(f"Erro ao reprojetar centróides para geográfico: {e}")
            return pizzeria_nodes, []
        
        # Verificar os tipos de geometria após reprojeção
        valid_geo = pizzeria_centroids_geo[pizzeria_centroids_geo.type == 'Point']
        invalid_geo = pizzeria_centroids_geo[pizzeria_centroids_geo.type != 'Point']
        if not invalid_geo.empty:
            print(f"{len(invalid_geo)} centróides inválidos removidos.")
            pizzeria_centroids_geo = valid_geo
        
        # Extrair coordenadas geográficas
        pizzeria_coords_geo = [(point.y, point.x) for point in pizzeria_centroids_geo]
        
        return pizzeria_nodes, pizzeria_coords_geo

def calculate_routes(G, origin_node, target_nodes, algorithms=['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra']):
    """
    Calcula as rotas mais curtas do nó de origem para cada nó de destino usando os algoritmos especificados.
    """
    routes = {alg: [] for alg in algorithms}
    times = {alg: [] for alg in algorithms}
    
    for alg in algorithms:
        for target in target_nodes:
            try:
                start_time = time.time()
                if alg == 'dijkstra':
                    route = nx.shortest_path(G, origin_node, target, weight='length')
                elif alg == 'astar':
                    route = nx.astar_path(
                        G, 
                        origin_node, 
                        target, 
                        weight='length', 
                        heuristic=lambda u, v: ox.distance.euclidean(
                            G.nodes[u]['y'], G.nodes[u]['x'], 
                            G.nodes[v]['y'], G.nodes[v]['x']
                        )
                    )
                elif alg == 'bellman_ford':
                    route = nx.bellman_ford_path(G, origin_node, target, weight='length')
                elif alg == 'bidirectional_dijkstra':
                    # networkx não possui uma função direta para Dijkstra bidirecional que retorna apenas o caminho.
                    # Podemos usar a função bidirectional_dijkstra para obter o custo e construir o caminho manualmente.
                    path = nx.bidirectional_dijkstra(G, origin_node, target, weight='length')[1]
                    route = path
                else:
                    raise ValueError("Algoritmo não suportado.")
                end_time = time.time()
                routes[alg].append(route)
                times[alg].append(end_time - start_time)
            except nx.NetworkXNoPath:
                print(f"Nenhuma rota encontrada para o nó {target} usando {alg}.")
            except Exception as e:
                print(f"Erro ao calcular rota para o nó {target} usando {alg}: {e}")
    
    avg_times = {alg: (sum(times[alg]) / len(times[alg]) if times[alg] else 0) for alg in algorithms}
    return routes, avg_times

def routes_to_geojson(G, routes, alg='dijkstra'):
    """
    Converte rotas para o formato GeoJSON.
    """
    features = []
    for route in routes[alg]:
        line = LineString([(G.nodes[node]['x'], G.nodes[node]['y']) for node in route])
        feature = geojson.Feature(geometry=line, properties={"algorithm": alg})
        features.append(feature)
    feature_collection = geojson.FeatureCollection(features)
    return feature_collection

def plot_routes_geojson(G, origin_point_geo, routes, pizzeria_coords_geo, transformer, algorithms=['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra']):
    """
    Plota rotas usando GeoJSON no Folium.
    """
    m = folium.Map(location=origin_point_geo, zoom_start=13)
    
    # Adicionar marcador para a origem
    folium.Marker(
        location=origin_point_geo,
        popup="Origem",
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)
    
    # Adicionar marcadores para as pizzarias
    for idx, coord in enumerate(pizzeria_coords_geo):
        folium.Marker(
            location=coord,
            popup=f"Pizzaria {idx+1}",
            icon=folium.Icon(color='red', icon='cutlery')
        ).add_to(m)
    
    # Definir cores para os algoritmos
    color_map = {
        'dijkstra': 'green',
        'astar': 'purple',
        'bellman_ford': 'orange',
        'bidirectional_dijkstra': 'blue'
    }
    
    for alg in algorithms:
        # Convertir todas as rotas para geo
        features = []
        for route in routes[alg]:
            route_proj = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route]
            route_geo = [transformer.transform(x, y) for x, y in route_proj]
            route_geo_latlon = [(lon, lat) for lon, lat in route_geo]  # geojson expects (lon, lat)
            line = LineString(route_geo_latlon)
            feature = geojson.Feature(geometry=line, properties={"algorithm": alg})
            features.append(feature)
        feature_collection = geojson.FeatureCollection(features)
        
        # Adicionar ao mapa
        folium.GeoJson(
            feature_collection,
            name=f"Rotas {alg.replace('_', ' ').capitalize()}",
            style_function=lambda feature, alg=alg: {
                'color': color_map.get(alg, 'blue'),
                'weight': 5,
            },
            tooltip=f"Rota {alg.replace('_', ' ').capitalize()}"
        ).add_to(m)
    
    folium.LayerControl().add_to(m)
    
    return m

def plot_single_route(G, origin_point_geo, route, transformer, alg='dijkstra'):
    """
    Plota uma única rota em um mapa interativo usando Folium.
    """
    # Converter as coordenadas projetadas para geográficas
    route_proj = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route]
    route_geo = [transformer.transform(x, y) for x, y in route_proj]
    route_geo_latlon = [(lat, lon) for lon, lat in route_geo]
    
    # Obter a coordenada do destino
    destination_node = route[-1]
    destination_proj = (G.nodes[destination_node]['x'], G.nodes[destination_node]['y'])
    destination_geo = transformer.transform(*destination_proj)
    destination_latlon = (destination_geo[1], destination_geo[0])
    
    # Definir cor com base no algoritmo
    color_map = {
        'dijkstra': 'green',
        'astar': 'purple',
        'bellman_ford': 'orange',
        'bidirectional_dijkstra': 'blue'
    }
    route_color = color_map.get(alg, 'blue')
    
    # Criar mapa centrado na origem
    m = folium.Map(location=origin_point_geo, zoom_start=13)
    
    # Adicionar marcador para a origem
    folium.Marker(
        location=origin_point_geo,
        popup="Origem",
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)
    
    # Adicionar marcador para o destino
    folium.Marker(
        location=destination_latlon,
        popup="Destino",
        icon=folium.Icon(color='red', icon='cutlery')
    ).add_to(m)
    
    # Adicionar a rota
    folium.PolyLine(
        route_geo_latlon, 
        color=route_color, 
        weight=5,  # Aumentar peso para melhor visibilidade
        opacity=1, 
        popup=f"Rota {alg.replace('_', ' ').capitalize()}"
    ).add_to(m)
    
    return m

def plot_routes_subset(G, origin_point_geo, routes, pizzeria_coords_geo, transformer, algorithms=['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra'], limit=10):
    """
    Plota as rotas calculadas em camadas separadas no mapa interativo usando Folium,
    limitando o número de rotas por algoritmo para 'limit'.
    """
    # Criar mapa centrado na origem
    m = folium.Map(location=origin_point_geo, zoom_start=13)
    
    # Adicionar marcador para a origem
    folium.Marker(
        location=origin_point_geo,
        popup="Origem",
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)
    
    # Adicionar marcadores para as pizzarias
    for idx, coord in enumerate(pizzeria_coords_geo):
        folium.Marker(
            location=coord,
            popup=f"Pizzaria {idx+1}",
            icon=folium.Icon(color='red', icon='cutlery')
        ).add_to(m)
    
    # Definir cores para os algoritmos
    color_map = {
        'dijkstra': 'green',
        'astar': 'purple',
        'bellman_ford': 'orange',
        'bidirectional_dijkstra': 'blue'
    }
    
    # Adicionar rotas em camadas separadas, limitando o número de rotas
    for alg in algorithms:
        layer = folium.FeatureGroup(name=f"Rotas {alg.replace('_', ' ').capitalize()}")
        for route in routes[alg][:limit]:
            try:
                # Obter as coordenadas projetadas dos nós da rota
                route_proj = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route]
                
                # Converter para geográficas
                route_geo = [transformer.transform(x, y) for x, y in route_proj]
                
                # Rearranjar para (lat, lon)
                route_geo_latlon = [(lat, lon) for lon, lat in route_geo]
                
                folium.PolyLine(
                    route_geo_latlon, 
                    color=color_map.get(alg, 'blue'), 
                    weight=5,  # Aumentar peso
                    opacity=1,  # Aumentar opacidade
                    popup=f"Rota {alg.replace('_', ' ').capitalize()}"
                ).add_to(layer)
            except Exception as e:
                print(f"Erro ao plotar rota {alg}: {e}")
        layer.add_to(m)
    
    # Adicionar controle de camadas
    folium.LayerControl().add_to(m)
    
    return m

# Exemplo de uso
if __name__ == "__main__":
    # Definir endereço de origem
    address = "Rua Augusta, São Paulo, Brasil"  # Substitua pelo endereço desejado
    try:
        origin_point = geocode_address(address)
        print(f"Coordenadas de Origem: {origin_point}")
    except ValueError as ve:
        print(ve)
        sys.exit()
    
    # Definir raio de busca em metros
    radius = 5000  # 5 km
    
    # Baixar grafo de ruas
    print("Baixando dados de ruas do OSM...")
    try:
        G = ox.graph_from_point(origin_point, dist=radius, network_type='drive', simplify=True)
        # Reprojetar o grafo para um CRS projetado (por exemplo, UTM)
        G_projected = ox.project_graph(G)
        print("Grafo de ruas carregado e reprojetado.")
    except Exception as e:
        print(f"Erro ao baixar ou processar o grafo: {e}")
        sys.exit()
    
    # Obter o CRS projetado
    crs_projected = G_projected.graph['crs']
    
    # Definir o transformador
    transformer = Transformer.from_crs(crs_projected, "epsg:4326", always_xy=True)
    
    # Converter as coordenadas da origem para o CRS projetado
    origin_gdf = gpd.GeoDataFrame(
        {'geometry': [Point(origin_point[1], origin_point[0])]},  # (lon, lat)
        crs='epsg:4326'  # CRS geográfico
    )
    origin_projected = origin_gdf.to_crs(crs_projected)
    origin_x_proj, origin_y_proj = origin_projected.geometry.x[0], origin_projected.geometry.y[0]
    
    # Encontrar nó de origem no grafo projetado
    try:
        origin_node = ox.distance.nearest_nodes(G_projected, X=origin_x_proj, Y=origin_y_proj)
        origin_node_coords = (G_projected.nodes[origin_node]['y'], G_projected.nodes[origin_node]['x'])
        print(f"Nó de Origem: {origin_node}")
        print(f"Coordenadas do Nó de Origem: {origin_node_coords}")
    except Exception as e:
        print(f"Erro ao encontrar o nó de origem: {e}")
        sys.exit()
    
    # Buscar pizzarias
    pizzeria_nodes, pizzeria_coords_geo = get_pizza_pois(G_projected, origin_point, radius=radius, cuisine='pizza')
    if not pizzeria_nodes:
        sys.exit()
    
    # Calcular rotas
    algorithms = ['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra']
    routes, avg_times = calculate_routes(G_projected, origin_node, pizzeria_nodes, algorithms=algorithms)
    
    # Exibir tempos médios
    for alg, avg_time in avg_times.items():
        print(f"Tempo Médio {alg.replace('_', ' ').capitalize()}: {avg_time:.6f} segundos")
    
    # Verificar número de rotas
    for alg in algorithms:
        print(f"Total de rotas para {alg}: {len(routes[alg])}")
    
    # Inspecionar as primeiras 3 rotas de cada algoritmo
    for alg in algorithms:
        if routes[alg]:
            print(f"\nPrimeiras 3 rotas para {alg.replace('_', ' ').capitalize()}:")
            for i, route in enumerate(routes[alg][:3], start=1):
                route_coords = [(G_projected.nodes[node]['y'], G_projected.nodes[node]['x']) for node in route]
                print(f" Rota {i} ({alg.replace('_', ' ').capitalize()}): {route_coords[:3]} ... {route_coords[-3:]}")
        else:
            print(f"\nNenhuma rota encontrada para {alg}.")
    
    # Plotar as primeiras 10 rotas de cada algoritmo
    m = plot_routes_subset(G_projected, origin_point, routes, pizzeria_coords_geo, transformer, algorithms=algorithms, limit=10)
    
    # Salvar o mapa em um arquivo HTML
    m.save('mapa_rotas_subset.html')
    print("Mapa salvo como 'mapa_rotas_subset.html'. Abra-o no seu navegador para visualização.")
    
    # Exibir mapa no Jupyter
    display(m)
    
    # Plotar uma única rota de cada algoritmo para teste
    for alg in algorithms:
        if routes[alg]:
            test_route = routes[alg][0]
            m_test = plot_single_route(G_projected, origin_point, test_route, transformer, alg=alg)
            display(m_test)
        else:
            print(f"Não há rotas para o algoritmo {alg}.")
    
    # Plotar rotas usando GeoJSON
    m_geojson = plot_routes_geojson(G_projected, origin_point, routes, pizzeria_coords_geo, transformer, algorithms=algorithms)
    m_geojson.save('mapa_rotas_geojson.html')
    print("Mapa GeoJSON salvo como 'mapa_rotas_geojson.html'. Abra-o no seu navegador para visualização.")
    display(m_geojson)


Coordenadas de Origem: (-22.1371202, -51.384717)
Baixando dados de ruas do OSM...
Grafo de ruas carregado e reprojetado.
